RAG WITH FHIR DATA LEVERAGING KNOWLEDGE GRAPHS 

In [1]:
from neo4j import GraphDatabase
import json
import os
from dotenv import load_dotenv


load_dotenv()
# Set up Neo4j connection
uri = os.getenv("NEO4J_URI", "")
username = os.getenv("NEO4J_USERNAME", "")
password = os.getenv("NEO4J_PASSWORD", "")
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to remove 'text.display' fields
def remove_text_display(data):
    for entry in data.get("entry", []):
        resource = entry.get("resource", {})
        if "text" in resource and "display" in resource["text"]:
            del resource["text"]["display"]
    return data

# Load and preprocess FHIR JSON file
with open(r"C:\Users\STORM\Documents\GitHub\RAGwithFHIR\Dataset\Bart73_King743_26b2b916-50a8-2b98-71b0-3150050312c8.json", "r") as file:
    fhir_data = json.load(file)

# Clean the data by removing 'text.display' fields
cleaned_data = remove_text_display(fhir_data)

# Functions for Neo4j database interactions

# 1. Clear the database
def clear_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")

# 2. Load FHIR data using CyFHIR
def load_bundle(tx, json_string):
    tx.run("CALL cyfhir.bundle.load($json, {validation: true, version: 'R4'})", json=json_string)

def fix_telecom(tx):
    tx.run("""MATCH (p)-[:telecom]->(ph)
                WITH ph, ph.use AS use, ph.value AS value
                CREATE (number:Telecom {use: use, value: value, code: use + " " + value})
                MERGE (ph)-[:coding]->(number)
                RETURN ph, number
           """)


# Execute script
with driver.session() as session:
    # Clear the database
    session.write_transaction(clear_database)
    
    # Load the cleaned FHIR data as a JSON string
    cleaned_json_string = json.dumps(cleaned_data)
    session.write_transaction(load_bundle, cleaned_json_string)
    session.write_transaction(fix_telecom)
    # Add indexes
    # session.write_transaction(add_indexes)
    
    # Create relationships for structured knowledge graph
    # session.write_transaction(create_relationships)

driver.close()


C:\Users\STORM\AppData\Local\Temp\ipykernel_21636\35975122.py:51: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(clear_database)
C:\Users\STORM\AppData\Local\Temp\ipykernel_21636\35975122.py:55: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(load_bundle, cleaned_json_string)
C:\Users\STORM\AppData\Local\Temp\ipykernel_21636\35975122.py:56: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(fix_telecom)


In [2]:
driver = GraphDatabase.driver(uri, auth=(username, password))

# Define queries
queries = [
    "MATCH (n:resource {resourceType: 'Condition'}) SET n:Condition",
    "MATCH (n:resource {resourceType: 'Condition'}) SET n:Embeddable",
    "MATCH (n:resource {resourceType: 'Observation'}) SET n:Observation",
    "MATCH (n:resource {resourceType: 'Observation'}) SET n:Embeddable",
    "MATCH (n:resource {resourceType: 'Medication'}) SET n:Medication",
    "MATCH (n:resource {resourceType: 'Medication'}) SET n:Embeddable",
    "MATCH (n:resource {resourceType: 'Patient'}) SET n:Patient",
    "MATCH (n:resource {resourceType: 'Patient'}) SET n:Embeddable",
    "MATCH (n:resource {resourceType: 'MedicationRequest'}) SET n:MedicationRequest",
    "MATCH (n:resource {resourceType: 'MedicationRequest'}) SET n:Embeddable",

]

# Function to run each query
def tag_resources(driver, queries):
    with driver.session() as session:
        for query in queries:
            session.run(query)
            # print(f"Executed: {query}")

# Run the tagging queries
tag_resources(driver, queries)

# Close the driver connection
driver.close()

Creating embedding text for patient node

In [3]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(username, password))

def create_patient_embedding(driver):
    query = """
        MATCH (p:Patient)
        OPTIONAL MATCH (p)-[:address]->(a)
        OPTIONAL MATCH (p)-[:maritalStatus]->(m)
        OPTIONAL MATCH (p)-[:name]->(n)
        RETURN 
            n.given as name,
            n.family as lname,
            p.gender AS gender,
            p.birthDate AS birthdate,
            a.city as city,
            a.country as country,
            a.state as state,
            a.line as line,
            m.text as marrigialStatus
    """

    # Run the query and process results
    with driver.session() as session:
        result = session.run(query)
        patient_info = []
        
        for record in result:
            # Debugging: print the raw record to see what is returned
            # print("Raw record:", record)
            
            # Collect data from each record
            name = record.get("name", "Unknown")
            lname = record.get("lname", "Unknown")
            gender = record.get("gender", "Not specified")
            birthdate = record.get("birthdate", "Unknown")
            marital_status = record.get("marrigialStatus", "Unknown")
            city = record.get("city", "Unknown")
            country = record.get("country", "Unknown")
            state = record.get("state", "Unknown")
            line = record.get("line", "Unknown")
            
            # Debugging: check the values of fields
            # print(f"Name: {name}, Line: {line}, City: {city}, State: {state}, Country: {country}")
            
            # Combine into one embedding text
            address = f"{line}, {city}, {state}, {country}" if line != "Unknown" else f"{city}, {state}, {country}"
            embedding_text = f"This is information about the patient, patient's Name: {name} {lname}, patient's Address: {address}, patient's Gender: {gender}, patient's Birthdate: {birthdate}, patient's Marital Status: {marital_status}"
            patient_info.append(embedding_text)

        update_query = """
            MATCH (p:Patient)

            SET p.embeddingText = $embedding_text
        """
        session.run(update_query, embedding_text=embedding_text)

        return patient_info

# Get the patient embedding text
patient_embeddingText = create_patient_embedding(driver)




In [4]:
from neo4j import GraphDatabase

# Initialize the Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

def create_patient_embedding(driver):
    query = """
        MATCH (c:Condition)
        OPTIONAL MATCH (c)-[:code]->(t)
        RETURN 
            c.recordedDate AS date,
            t.text AS text,
            id(c) AS nodeId
    """

    # Run the query and process results
    with driver.session() as session:
        result = session.run(query)
        patient_info = []
        
        for record in result:
            # Extract the data safely
            date = record.get("date", "Unknown")
            text = record.get("text", "Unknown")
            node_id = record["nodeId"]

            # Construct the embedding text
            embedding_text = f"The patient had the condition: {text}, and it was recorded on this date: {date}"

            # Store the embedding text in the corresponding node
            update_query = """
                MATCH (c:Condition)
                WHERE id(c) = $node_id
                SET c.embeddingText = $embedding_text
            """
            session.run(update_query, node_id=node_id, embedding_text=embedding_text)

            # Collect the embedding text for debugging or further use
            patient_info.append(embedding_text)

        return patient_info

# Get the patient embedding text
patient_embedding_texts = create_patient_embedding(driver)

# Debugging: Print all generated embedding texts
# for text in patient_embedding_texts:
#     print(text)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 7, column: 13, offset: 161} for query: '\n        MATCH (c:Condition)\n        OPTIONAL MATCH (c)-[:code]->(t)\n        RETURN \n            c.recordedDate AS date,\n            t.text AS text,\n            id(c) AS nodeId\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 23, offset: 59} for query: '\n                MATCH (c:Condition)\n                WHERE id(c) = $node_id\n                SET c.embeddingText = $embeddi

In [ ]:
from neo4j import GraphDatabase

# Initialize the Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

def create_patient_embedding(driver):
    query = """
        MATCH (o:Observation)
                OPTIONAL MATCH (o)-[:code]->(t)
                OPTIONAL MATCH (o)-[:valueQuantity]->(v)
                RETURN 
                    o.status as status,
                    t.text AS text,
                    v.unit as unit,
                    v.value as value,
                    id(o) AS nodeId
    """

    # Run the query and process results
    with driver.session() as session:
        result = session.run(query)
        patient_info = []
        
        for record in result:
            # Extract the data safely
            status = record.get("status", "Unknown")
            text = record.get("text", "Unknown")
            unit = record.get("unit", "Unknown")
            value = record.get("value", "Unknown")
            node_id = record["nodeId"]

            # Construct the embedding text
            embedding_text = f"The patient had an observation for : {text}, and it was recorded : {value}{unit} with the status of the observation being {status}"

            # Store the embedding text in the corresponding node
            update_query = """
                MATCH (o:Observation)
                WHERE id(o) = $node_id
                SET o.embeddingText = $embedding_text
            """
            session.run(update_query, node_id=node_id, embedding_text=embedding_text)

            # Collect the embedding text for debugging or further use
            patient_info.append(embedding_text)

        return patient_info

# Get the patient embedding text
patient_embedding_texts = create_patient_embedding(driver)

# Debugging: Print all generated embedding texts
# for text in patient_embedding_texts:
#     print(text)


In [6]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(username, password))

def create_patient_embedding(driver):
    query = """
        MATCH (m:Medication)
                OPTIONAL MATCH (m)-[:code]->(t)
                RETURN 
                    m.status as status,
                    t.text AS text,
                    id(m) AS nodeId
    """

    # Run the query and process results
    with driver.session() as session:
        result = session.run(query)
        patient_info = []
        
        for record in result:
            # Debugging: print the raw record to see what is returned
            # print("Raw record:", record)
            
            # Collect data from each record
            status = record.get("status", "Unknown")
            text = record.get("text", "Unknown")
            node_id = record["nodeId"]

            
            # Combine into one embedding text
            embedding_text = f"Medication given to patient with the name: {text} and the status: {status}"
            patient_info.append(embedding_text)

            update_query = """
                MATCH (m:Medication)
                WHERE id(m) = $node_id
                SET m.embeddingText = $embedding_text
            """
            session.run(update_query, node_id=node_id, embedding_text=embedding_text)

        return patient_info

# Get the patient embedding text
patient_embeddingText = create_patient_embedding(driver)




Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 7, column: 21, offset: 198} for query: '\n        MATCH (m:Medication)\n                OPTIONAL MATCH (m)-[:code]->(t)\n                RETURN \n                    m.status as status,\n                    t.text AS text,\n                    id(m) AS nodeId\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 23, offset: 60} for query: '\n                MATCH (m:Medication)\n                WHERE id(m) = $node_id\n        

In [7]:
# from neo4j import GraphDatabase

# driver = GraphDatabase.driver(uri, auth=(username, password))

# def create_patient_embedding(driver):
#     query = """
#         MATCH (m:MedicationRequest) 
#         optional match (m)-[:requester]->(r)
#         optional match (m)-[:medicationCodeableConcept]->(mc)
#         optional match (m)-[:dosageInstruction]->(d)
#         RETURN 
#             m.status as status,
#             r.display as requester,
#             d.text as instrocution,
#             mc.text as medication,
#             id(m) as nodeId
#     """

#     # Run the query and process results
#     with driver.session() as session:
#         result = session.run(query)
#         patient_info = []
        
#         for record in result:
#             # Debugging: print the raw record to see what is returned
#             # print("Raw record:", record)
            
#             # Collect data from each record
#             status = record.get("status", "Unknown")
#             instrocution = record.get("text", "Unknown")
#             requester = record.get("requester", "Unknown")
#             medication = record.get("medication", "Unknown")
#             node_id = record["nodeId"]

            
#             # Combine into one embedding text
#             embedding_text = f"The medication: {medication} was requested for the patient from {requester} with instrocution {instrocution}, the status of the medication request is {status}."
#             patient_info.append(embedding_text)

#             update_query = """
#                 MATCH (m:MedicationRequest)
#                 WHERE id(m) = $node_id
#                 SET m.embeddingText = $embedding_text
#             """
#             session.run(update_query, node_id=node_id, embedding_text=embedding_text)

#         return patient_info

# # Get the patient embedding text
# patient_embeddingText = create_patient_embedding(driver)




Vector index from graph

In [8]:
from langchain.graphs import Neo4jGraph
from langchain.vectorstores.neo4j_vector import Neo4jVector

from langchain.embeddings import HuggingFaceEmbeddings
import langchain
langchain.debug = True
# Specify the model from Hugging Face, e.g., "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_index = None
# Create the vector index from the existing Neo4j graph
vector_index = Neo4jVector.from_existing_graph(
    hf_embeddings,
    url=uri,
    username=username,
    password=password,
    index_name='FHIR_INDEX',
    node_label="Embeddable",  # Replace with the actual label of your nodes
    text_node_properties=['embeddingText'],  # Properties to embed
    embedding_node_property='embedding',  # Property to store embeddings in Neo4j
)

# print(vector_index)
# question = "what condition does the patient have"

# response = vector_index.similarity_search(question, k=100)
# print(response)


C:\Users\STORM\AppData\Local\Temp\ipykernel_21636\3560383851.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\STORM\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
print(vector_index.index_name)
print(vector_index.embedding_node_property)
print(vector_index.node_label)


FHIR_INDEX
embedding
Embeddable


In [69]:
from langchain.llms import Ollama
from pprint import pprint

langchain.debug = False



def rag(query):
    return vector_index.similarity_search(query, k=5)

# context = rag()

# pprint(context)

def llm_query(context, query):
    prompt_template = """
    You are a helpful assistant with knowledge of FHIR data, the current year is 2024. Use the following context to answer the query:

    Context:
    {context}

    Question:
    {query}

    Provide a concise and accurate answer.
    """
    prompt = prompt_template.format(context=context, query=query)



    # Initialize the LLM
    llm = Ollama(model="llama3.2", temperature=0.7)

    # Generate the answer
    answer = llm(prompt)
    return(answer)

# print("LLM Response:", llm_query(context, query))


Contextualize the search 


In [30]:
ids = []
for rs in context:
    ids.append(rs.metadata['_resourceId'])

# pprint(ids)

contextualize_query_template = """
    match (n)-[r]->(c)
    match (c)-[:coding]->(t)
    where n._resourceId = '{rid}' and type(r) <> "code" AND n.resourceType is not null
    with type(r) as relationshipType, t.code as code, {{}} as metadata, 1.0 as score
    return relationshipType + ': ' + code as text
"""

contextualized_output = []

def run_query_and_store_output(tx, query):
    # Execute the query and fetch results
    result = tx.run(query)
    # Convert results to a list of dictionaries
    return [record.data() for record in result]

with driver.session() as session:
    for id in ids:
        # Format the query for the current ID
        contextualize_query = contextualize_query_template.format(rid=id)
        # Run the query and store the results
        data = session.read_transaction(run_query_and_store_output, contextualize_query)
        # print(data)
        data.append({"id": id})
        # Pair the ID with its results
        contextualized_output.append(data)

driver.close()

pprint(contextualized_output)

[[{'text': 'telecom: home 555-105-5326'},
  {'text': 'maritalStatus: M'},
  {'id': '26b2b916-50a8-2b98-71b0-3150050312c8'}],
 [{'text': 'clinicalStatus: active'},
  {'text': 'verificationStatus: confirmed'},
  {'text': 'category: encounter-diagnosis'},
  {'id': '136ccc15-a5e0-69bb-5d0f-5d0a8074321b'}],
 [{'text': 'clinicalStatus: resolved'},
  {'text': 'verificationStatus: confirmed'},
  {'text': 'category: encounter-diagnosis'},
  {'id': '900d6855-c8f3-099f-401b-fdc25e577c73'}],
 [{'text': 'clinicalStatus: active'},
  {'text': 'verificationStatus: confirmed'},
  {'text': 'category: encounter-diagnosis'},
  {'id': 'a46b5767-7834-b56b-6c48-0fa6c9a558b3'}],
 [{'text': 'clinicalStatus: active'},
  {'text': 'verificationStatus: confirmed'},
  {'text': 'category: encounter-diagnosis'},
  {'id': 'be2d15e4-7dbd-1205-501d-97cfdb0db340'}]]


C:\Users\STORM\AppData\Local\Temp\ipykernel_21636\1847563737.py:23: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\STORM\AppData\Local\Temp\ipykernel_21636\1847563737.py:28: DeprecationWarning: read_transaction has been renamed to execute_read
  data = session.read_transaction(run_query_and_store_output, contextualize_query)


In [73]:
def add_context(context):
    texts = []
    for doc in context:
        for extra in contextualized_output:
            if(doc.metadata['_resourceId'] == extra[-1]['id']):
                for i in range(len(extra)-1) :
                    texts.append(extra[i]['text'])
                    doc.page_content += f" {extra[i]['text']}"
    return texts
    



In [81]:
query = "what is the patient name and telecom"

context = rag(query)
print("\nRAG OUTPUT:")
pprint(context)

print("\nLLM output with only RAG:")
print(llm_query(context, query))

extras = add_context(context)
print("\nExtras:")
pprint(extras)

print("\nLLM output with RAG and contextualized data:")
print(llm_query(context, query))


RAG OUTPUT:
[Document(metadata={'_isArray': False, 'multipleBirthBoolean': False, 'gender': 'male', 'birthDate': '1991-06-13', '_resourceId': '26b2b916-50a8-2b98-71b0-3150050312c8', 'resourceType': 'Patient'}, page_content="\nembeddingText: This is information about the patient, patient's Name: [Bart73, Caleb651] King743, patient's Address: [270 Johns Brook Suite 56], North Plymouth, MA, US, patient's Gender: male, patient's Birthdate: 1991-06-13, patient's Marital Status: Married"),
 Document(metadata={'_isArray': False, 'onsetDateTime': '2009-08-06T04:44:50+00:00', 'recordedDate': '2009-08-06T04:44:50+00:00', '_resourceId': '136ccc15-a5e0-69bb-5d0f-5d0a8074321b', 'resourceType': 'Condition'}, page_content='\nembeddingText: The patient had the condition: Received higher education (finding), and it was recorded on this date: 2009-08-06T04:44:50+00:00'),
 Document(metadata={'_isArray': False, 'onsetDateTime': '2016-08-18T05:01:59+00:00', 'abatementDateTime': '2019-08-22T04:45:13+00:00'

In [12]:
resourceId = context[0].metadata['_resourceId']
print(resourceId)
# contextualize_query_patient = f"""
#     MATCH (n)-[:telecom]->(p)
#     WHERE n.`_resourceId` = '{resourceId}'
#     WITH p.value + ' ' + p.use AS self, {{}} AS metadata, 1.0 AS score
#     RETURN self AS text, score, metadata
# """



contextualized_vectorstore = Neo4jVector.from_existing_index(
    hf_embeddings,
    url=uri,
    username=username,
    password=password,
    index_name="FHIR_INDEX",

)

d0d87024-5d8c-9772-a776-817eefb627fe


In [13]:
print(query)
response = contextualized_vectorstore.similarity_search("patient info", k=1, retrieval_query=contextualize_query_condition)
pprint(response)

did the patient have neck injury


NameError: name 'contextualize_query_condition' is not defined